# Human in the Loop

Your durable research application now survives crashes and automatically retries failures. But there's a critical gap: **it runs completely autonomously**.

Imagine this scenario: Your AI generates research on "best travel spots for summer 2025", your application generates a PDF, and sends it to your client—all automatically.

Then your client calls: "I want cheaper options!"

Your application had no way to pause and ask for clarification or approval. It just executed straight through to completion.

**Real-world AI applications need human interaction**—to provide feedback, approve decisions, or clarify requirements. But adding human input to automated workflows introduces new challenges: What if the user's browser crashes while they're reviewing? What if they close the tab and come back later? How do you prevent losing expensive LLM work while waiting for approval?

In this section, we'll solve these problems by making your AI application interactive and durable.

## Setup Notebook

Run the following code blocks to install various packages and tools necessary to run this notebook

**Be sure to add your .env file again. It doesn't persist across notebooks or sesions**

In [ ]:
# We'll first install the necessary packages for this workshop.

%pip install --quiet temporalio litellm reportlab python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
# Create .env file
with open(".env", "w") as fh:
  fh.write("LLM_API_KEY = YOUR_API_KEY\nLLM_MODEL = openai/gpt-4o")

  # Now open the file and replace YOUR_API_KEY with your API key.

In [ ]:
# Load environment variables and configure LLM settings
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Get LLM_API_KEY environment variable and print it to make sure that your .env file is properly loaded.
LLM_MODEL = os.getenv("LLM_MODEL", "openai/gpt-4o")
LLM_API_KEY = os.getenv("LLM_API_KEY", None)
print("LLM API Key", LLM_API_KEY)

In [2]:
# Mermaid renderer, run at the beginning to setup rendering of diagrams
import base64
from IPython.display import Image, display

def render_mermaid(graph_definition):
    """
    Renders a Mermaid diagram in Google Colab using mermaid.ink.

    Args:
        graph_definition (str): The Mermaid diagram code (e.g., "graph LR; A-->B;").
    """
    graph_bytes = graph_definition.encode("ascii")
    base64_bytes = base64.b64encode(graph_bytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

In [ ]:
# This allows us to run the Temporal Asyncio event loop within the event loop of Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

In [ ]:
 # Running this will download the Temporal CLI

!curl -sSf https://temporal.download/cli.sh | sh

## Make Sure Your Temporal Web UI is Running

1. You should have the Temporal Server running in your terminal (run `temporal server start-dev` if not).
2. Then in your `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon to open the Temporal Web UI.

## Review the Previous Workflow

Let's quickly review the previous Workflow below to refresh your memory:

### Data Models

* Temporal recommends passing data to and from Activities and Workflows as a single object.
* Use a dataclass for this.

In [ ]:
# Run this code block to load it into the program
from dataclasses import dataclass

@dataclass
class LLMCallInput:
  prompt: str
  llm_api_key: str
  llm_model: str

@dataclass
class PDFGenerationInput:
  content: str
  filename: str = "research_pdf.pdf"

@dataclass
class GenerateReportInput:
    prompt: str
    llm_api_key: str
    llm_research_model: str = "openai/gpt-4o"
    llm_image_model: str = "dall-e-3"

@dataclass
class GenerateReportOutput:
    result: str

### Activities

* An Activity is a function/method that is prone to failure and/or non-deterministic.
* Temporal requires all non-deterministic code be run in an Activity
* Activities retry over and over until they succeed or until your customized retry or timeout configuration is hit.
* You define an Activity by adding the `@activity.defn` decorator.

In [ ]:
# Run this code block to load it into the program
from temporalio import activity
from litellm import completion, ModelResponse
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

@activity.defn
def llm_call(input: LLMCallInput) -> ModelResponse:
    response = completion(
      model=input.llm_model,
      api_key=input.llm_api_key,
      messages=[{ "content": input.prompt,"role": "user"}]
    )
    return response

@activity.defn
def create_pdf(input: PDFGenerationInput) -> str:
    print("Creating PDF document...")

    doc = SimpleDocTemplate(input.filename, pagesize=letter)
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=24,
        spaceAfter=30,
        alignment=1
    )

    story = []
    title = Paragraph("Research Report", title_style)
    story.append(title)
    story.append(Spacer(1, 20))
    paragraphs = input.content.split('\n\n')
    for para in paragraphs:
        if para.strip():
          p = Paragraph(para.strip(), styles['Normal'])
          story.append(p)
          story.append(Spacer(1, 12))

    doc.build(story)

    print(f"SUCCESS! PDF created: {input.filename}")
    return input.filename

### Workflow

* Activities are orchestrated within a Temporal Workflow.
* Workflows must **not** make API calls, file system calls, or anything non-deterministic. That is what Activities are for.
* Workflows are async, and you define them as a class decorated with the `@workflow.defn` decorator.
* Every Workflow has a **single** entry point, which is an `async` method decorated with `@workflow.run`.

In [ ]:
# Run this code block to load it into the program
from datetime import timedelta
from temporalio import workflow

# sandboxed=False is a Notebook only requirement. You normally don't do this
@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:

        llm_call_input = LLMCallInput(prompt=input.prompt, llm_api_key=LLM_API_KEY, llm_model=LLM_MODEL)

        research_facts = await workflow.execute_activity(
            llm_call,
            llm_call_input,
            start_to_close_timeout=timedelta(seconds=30),
        )

        workflow.logger.info("Research complete!")

        # Uncomment to add delay
        # await workflow.sleep(timedelta(seconds=20))

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=10),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

## Human Interactions in Your AI Applications

While some AI applications may operate entirely autonomously, many require human intervention. They may provide input on launch or at various points throughout the execution.

Examples:
  a. Validation at critical decision points
  b. Final review before implementation
  c. Feedback loops

### Example: Customer Support Agent with Human Oversight

The Scenario:
Customer contacts support: "My order hasn't arrived and I need it urgently"

**How Human Interaction Works:**

1. **Initial Input** - Customer provides problem; human sets constraints like maximum refund amounts or escalation thresholds

2. **AI Analysis** - LLM checks order status and finds "delayed in transit," then proposes: "Process a $75 expedited replacement"

3. **Human Decision Point** - System pauses for approval. Human can:
   - Approve the proposed action
   - Modify the approach ("Offer refund instead")
   - Add context ("VIP customer - upgrade to overnight")

**Why This Matters:**
- **Cost control** - Expensive decisions require human approval
- **Security** - Prevent unauthorized refunds or account changes
- **Risk mitigation** - Human judgment for edge cases and exceptions
- **Compliance** - Legal/regulatory requirements for certain actions

In [3]:
diagram = """
graph LR
    U1[User Interaction optional] -.-> L[LLM makes decisions on what to do next]
    L --> A[Action does what the LLM decided]
    A --> L
    U2[User Interaction optional] -.-> A
"""
render_mermaid(diagram)

## Challenges in Non-Durable Human in the Loop Processes

While human interaction points are valuable for AI applications, implementing them reliably presents significant technical challenges. Without durable execution, human input can be lost during system failures, leading to unpredictable behavior.

**Consider the following scenario**

- A user needs to approve a transaction.
- As they are doing this, the website goes down.
- How do you mitigate this?
  - Do we notify the user to approve the payment again? (creating confusion since the user already 'approved')
  - Do we assume approval and risk processing an unauthorized payment?

Without durable processes, you're forced to choose between security, user experience, and reliability.

## Signals in Temporal

In Temporal, human interaction in Temporal systems is achieved through a [Temporal Signal](https://docs.temporal.io/encyclopedia/workflow-message-passing).

A Signal is a:
* Message sent asynchronously to a running Workflow Execution
* Used to to change the state and control the flow of a Workflow Execution.

*Take 2 minutes to discuss with your neighbor when we might use a Signal, then we'll share answers*



## Example Signal Usage

1. The user initiates a workflow with an initial request
2. The workflow processes the request and determines what information or approval is needed
3. The workflow pauses and waits for user input via Signal, such as:
    - Additional information or clarification
    - Permission to proceed with an action
    - Selection between multiple options
4. The user sends a Signal with their response
5. The workflow resumes execution based on the Signal received
6. Steps repeat as needed until the workflow completes its task

## Durably Storing Human Interactions

Let's go back to the user approving a payment example now. With Temporal, when the user clicks "approve" in the finance portal, the approval decision gets durably stored.

The user can close the browser, go to lunch, and the Workflow will continue running in the background.

If the payment gateway times out, returns an error or becomes unavailable, Temporal automatically retries the payment step. IT does not need to re-ask the user for approval, because that decision is already durably stored in the Workflow state.

- **No duplicate work** (user does not have to re-approve the same expense)
- **No lost approvals** (Signal persists and processing resumes automatically when system recovers)
- **No manual intervention** (does not need to manually reconcile failed payments or investigate whether an expense was actually approved)
- **Reliable processing** (business can count on approved expenses being paid)

## Developing Signals in Temporal

There are two steps for adding support for a Signal to your Workflow code:

1. **Defining the Signal** - You specify the name and data structure used by Temporal Clients when sending the Signal.
2. **Handling the Signal** - You write code that will be invoked when the Signal is received from a Temporal Client.

## Demo (Expand for instructor notes or to run on your own)
<!--
1. Clone this repository: `https://github.com/temporalio/edu-ai-workshop-agentic-loop`. The instructions will also be in the README.
2. From the `src/module_one_03_human_in_the_loop` directory, run `app.py` with `python app.py`.
3. In one terminal window, run your Worker with `python worker.py`.
4. In another terminal window, execute your Workflow with `python starter.py`.
5. You'll be prompted to enter a research topic or question in the CLI.
6. Once you do, you'll be prompted with the ability to Signal or Query the Workflow.
7. Type 'query' and you'll see the output in the terminal window where you started your Workflow Execution.
8. Time to demonstrate Signals. Back in the terminal window when you started your Workflow Execution, you'll see that you are prompted to choose one of the two options:
    a. Approve of this research and if you would like it to create a PDF (type 'keep' to send a Signal to the Workflow to create the PDF).
    b. Modify the research by adding extra info to the prompt (type 'edit' to modify the prompt and send another Signal to the Workflow to prompt the LLM again).
9. Demonstrate the modification by typing `edit`.
10. Enter additional instructions (e.g.: "turn this into a poem") and see the new output in the terminal window by typing `query` again.
11. Finally, show that you can keep changing the execution path of your Workflow Execution by typing `keep`. Show that the PDF has appeared in your `module_one_03_human_in_the_loop` directory.
-->

## Implementing a Signal

To add Signal support to your Workflow, you need to complete four key steps:

1. **Create a custom model for the Signal data** 
     - Define the structure of information that will be sent via Signal 
     - This model determines what information can be passed to your running Workflow 
2. **Initialize instance variables in your Workflow's `__init__` method** 
     - Add instance variables to store incoming Signal data
    - These variables persist throughout the Workflow's execution
    - Set default values that represent the "no Signal received yet state         
3. **Implement a Signal handler method decorated with `@workflow signal`**
    - Define a method that will be called when a Signal is received
    - This method updates your instance variables with the incoming Signal data
4. **React to the Signal in your Workflow logic** 
    - Use `workflow.wait_condition()` to pause execution until a Signal is received
    - Check the Signal data and branch your Workflow logic accordingly

## Let's Add a Signal to Our Workflow

We are going to create a Signal to send to our `GenerateReportWorkflow`. After generating the research, and before creating a PDF report, the user can review the research. The user has two choices to send to the Workflow:
  - Keep the research and generate a PDF report by signaling "keep"
  - Modify the research before generating a PDF report by signaling "edit"

### Create a Model

- Create a model for the Signal to be stored in
- Similar to Activities and Workflows, `dataclasses` are recommended here
- The model can be nested of other classes, such as `StrEnum`

In [ ]:
# This defines a string enum and a dataclass for handling user decisions in a Temporal workflow.
# We will send a `UserDecision` as a Signal to our Research Workflow letting the Workflow know if we want to keep or edit the research
# Or if we want to wait for further decision
# Run this code block to load it into the program
from enum import StrEnum

class UserDecision(StrEnum):
    KEEP = "KEEP"
    EDIT = "EDIT"
    WAIT = "WAIT"
    
@dataclass
class UserDecisionSignal: # A data structure to send user decisions via Temporal Signals
    decision: UserDecision
    additional_prompt: str = ""

## Handle the Signal

Signals are asynchronous messages sent to running workflows from external systems (like a user clicking a button or an API call). Once a Signal is received, your Workflow needs to process that data and take appropriate action based on the Signal handler function.

Remember, there are three main components:

1. **Store Signal State**: Instance variables to store incoming Signal data that persist across Workflow Executions and Replays.
2. **Signal Handler Method**: The handler is a method decorated with `@workflow.signal` that receives incoming Signals and updates your stored state.
3. **React to Signal in Workflow Logic**: After storing the signal, your workflow's main logic needs to check the state and react accordingly.

Let's first look at storing the Signal state.

### Storing the Signal state

Remember, the workflow needs a place to remember what Signals it has received:

```python
@workflow.defn
class GenerateReportWorkflow:
    def __init__(self):
        # Instance variable - persists across workflow execution
        self._user_decision = UserDecisionSignal(decision=UserDecision.WAIT)
```

- Can be a simple variable, or a Queue for handling many Signals
- Initialize with default values that indicates "no Signal received yet". In the example above, `WAIT` is the default state.

Now let's define a Signal handler.

## Defining a Signal Handler

A method within your Workflow class that receives and processes incoming Signals.

```python
@workflow.signal
async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
    self._user_decision = decision_data
```

In this example, when a client sends a Signal, this handler receives the `UserDecisionSignal` data and updates the `self._user_decision` in the Workflow.

In [ ]:
# Putting it Together
# Reflection Questions:
# 1. What's still missing for this Signal implementation to work properly?
# 2. How will the Workflow know when to check for a Signal?
from temporalio import workflow

@workflow.defn(sandboxed=False) # sandboxed=False is a Notebook only requirement. You normally don't do this)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        # Instance variable to store Signal data
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT) # UserDecision Signal starts with WAIT as the default state

    # Define the Signal handler
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update instance variable when Signal is received
        self._user_decision = decision_data

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
      self._current_prompt = input.prompt

      llm_call_input = LLMCallInput(
          prompt=self._current_prompt,
          llm_api_key=input.llm_api_key,
          llm_model=input.llm_research_model,
      )
      # rest of code here

## Waiting for a Signal

Now that we have a Signal handler to receive data, we need a way for the Workflow to pause and wait for that Signal to arrive. This is where `workflow.wait_condition()` comes in.

**How `workflow.wait_condition()` works:**
- Pauses Workflow execution until a specified condition becomes true
- Creates a durable checkpoint where the Workflow stops and waits
- Resumes execution only when the condition evaluates to `True`
- Optionally accepts a timeout parameter: `workflow.wait_condition(lambda: condition, timeout=timedelta(hours=24))` - waits until Signal received OR timeout elapsed, whichever happens first

**Benefits:**
- **Resource efficiency**: During waiting periods, the Workflow instance consumes no CPU or memory. The Worker only "wakes up" when a Signal is received or when replaying history.
- **Durability**: If systems crash while waiting, the Workflow resumes exactly where it left off when systems recover

In [ ]:
# Let's add a wait_condition to our code
# TODO: Within the `await workflow.wait_condition(lambda: )` code, 
# set the lambda to when `self._user_decision.decision` is not set to `UserDecision.WAIT`
from temporalio import workflow

@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        # Instance variable to store the Signal in
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT) # UserDecision Signal starts with WAIT as the default state

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(
            prompt=self._current_prompt,
            llm_api_key=input.llm_api_key,
            llm_model=input.llm_research_model,
        )

        continue_agent_loop = True

        while continue_agent_loop:
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            self._research_result = research_facts["choices"][0]["message"]["content"]

            # Waiting for Signal with user decision
            await workflow.wait_condition(lambda: ) # TODO set the lambda to when `self._user_decision.decision` is not set to `UserDecision.WAIT`

            # rest of code here

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "wait_condition_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

## Handle the Signal: React to Signal in Workflow Logic

After receiving a Signal, your Workflow's main execution logic must evaluate the Signal data and determine the appropriate response. This is where the Workflow's business logic intersects with human input.

In this example, we are branching execution based on Signal content.

```
if self._user_decision.decision == UserDecision.KEEP:
    continue_agent_loop = False
elif self._user_decision.decision == UserDecision.EDIT:
    # Modify prompt and reset for next iteration
    self._user_decision = UserDecisionSignal(decision=UserDecision.WAIT)
```

- If the Workflow receives `KEEP` as the `UserDecision`, then the Workflow exits the research loop and proceeds to PDF generation.
- If the Workflow receives `EDIT` as the `UserDecision`, then the Workflow incorporates any additional feedback into the prompt, updates the research parameters, and resets the Signal state back to WAIT so it can loop again to regenerate the research and wait for the next user decision.

In [ ]:
# Putting it together, we see we have set the Signal state, created the Signal handler method, 
# and have logic in the Workflow to how it will react to the Signal.
# Step 1. If the user chooses to edit the research, after they supply a new prompt,
# set the user decision back to WAIT for the next loop
# Step 2: Run this code block to load it into your program
from datetime import timedelta
from temporalio import workflow

@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        # Instance variable to store the Signal in
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT)
        self._research_result: str = ""

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(
            prompt=self._current_prompt,
            llm_api_key=input.llm_api_key,
            llm_model=input.llm_research_model,
        )

        continue_agent_loop = True

        while continue_agent_loop:
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            self._research_result = research_facts["choices"][0]["message"]["content"]

            # Waiting for Signal with user decision
            await workflow.wait_condition(lambda: self._user_decision.decision != UserDecision.WAIT)

            if self._user_decision.decision == UserDecision.KEEP:
                workflow.logger.info("User approved the research. Creating PDF...")
                continue_agent_loop = False
            elif self._user_decision.decision == UserDecision.EDIT:
                workflow.logger.info("User requested research modification.")
                if self._user_decision.additional_prompt != "":
                    self._current_prompt = (
                        f"{self._current_prompt}\n\nAdditional instructions: {self._user_decision.additional_prompt}"
                    )
                else:
                    workflow.logger.info("No additional instructions provided. Regenerating with original prompt.")
                llm_call_input.prompt = self._current_prompt

                # TODO Set the user decision back to WAIT for the next loop
                self._user_decision = UserDecisionSignal(decision=UserDecision.WAIT)

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename: str = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=20),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "putting_it_together_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Let's set up our Worker. Run this code block to load it into the program
import concurrent.futures
from temporalio.client import Client
from temporalio.worker import Worker

async def run_worker() -> None:
    # Create client connected to server at the given address
    client = await Client.connect("localhost:7233", namespace="default")

    # Run the Worker
    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as activity_executor:
        worker = Worker(
            client,
            task_queue="research",
            workflows=[GenerateReportWorkflow],
            activities=[llm_call, create_pdf],
            activity_executor=activity_executor
        )
        print(f"Starting the worker..")
        await worker.run()

print("Worker loaded!")

## Test the Workflow

Now that your Signal is implemented, let's run the Worker and start the Workflow.

### Run a Worker

As always, code won't execute if a Worker isn't running.

In [ ]:
import asyncio
worker = asyncio.create_task(run_worker())

### Start the Workflow

Now start the Workflow:

In [9]:
import uuid
from temporalio.client import Client

client = await Client.connect("localhost:7233", namespace="default")

print("Welcome to the Research Report Generator!")
prompt = input("Enter your research topic or question: ").strip()

if not prompt:
    prompt = "Give me 5 fun and fascinating facts about tardigrades. Make them interesting and educational!"
    print(f"No prompt entered. Using default: {prompt}")

# Asynchronous start of a Workflow
handle = await client.start_workflow(
    GenerateReportWorkflow.run,
    GenerateReportInput(prompt=prompt, llm_api_key=LLM_API_KEY),
    id=f"generate-research-report-workflow-{uuid.uuid4()}",
    task_queue="research",
)

print(f"Started workflow. Workflow ID: {handle.id}, RunID {handle.result_run_id}")

Welcome to the Research Report Generator!
No prompt entered. Using default: Give me 5 fun and fascinating facts about tardigrades. Make them interesting and educational!


NameError: name 'GenerateReportWorkflow' is not defined

## Observing Signals in the Web UI

Now, look at the Web UI. What do you observe?

_**To run the Temporal Server in this exercise environment**:
1. You should have the Temporal Server running in your terminal (run `temporal server start-dev` if not).
2. Then in your `Ports` tab on the bottom of this screen, find `8233` and click on the Globe icon to open the Temporal Web UI._

## Sending Signals

- We can now run the Workflow and send our Signal
- There are multiple ways to send a Signal
  - Using a Temporal Client in an SDK
  - Using the Web UI
  - Using the `temporal` cli
- For this example, we will use a Temporal Client.

## Sending a Signal with the Client

To send a Signal with the Temporal Client, we need to get a "handle" to a specific Workflow Execution, which will be used to interact with that Workflow.

We'll do this with the `get_workflow_handle` method.

```python
handle = client.get_workflow_handle(workflow_id)
```

With the handle on the Workflow Execution we want to Signal, we'll then pass in our Signal:

```python
signal_data = UserDecisionSignal(decision=UserDecision.KEEP)
await handle.signal("user_decision_signal", signal_data)
```

In [ ]:
# Let's send our Signal from the Client code
# Step 1: If the input decision was to edit the the prompt, send a Signal to your Workflow Handle with the new signal data
# Step 2: Run this code block to load it into the program
async def send_user_decision_signal(client: Client, workflow_id: str) -> None:
  handle = client.get_workflow_handle(workflow_id) # Get a handle on the Workflow Execution we want to send a Signal to.

  while True:
      print("\n" + "=" * 80)
      print(
          "Research is complete! The response will output in the terminal window with the Worker running. What would you like to do?"
      )
      print("1. Type 'keep' to approve the research and create PDF")
      print("2. Type 'edit' to modify the research")
      print("=" * 80)

      decision = input("Your decision (keep/edit): ").strip().lower()

      if decision in {"keep", "1"}:
          signal_data = UserDecisionSignal(decision=UserDecision.KEEP)
          await handle.signal("user_decision_signal", signal_data) # Send our Keep Signal to our Workflow Execution we have a handle on
          print("Signal sent to keep research and create PDF")
          break
      if decision in {"edit", "2"}:
          additional_prompt_input = input("Enter additional instructions for the research (optional): ").strip()
          additional_prompt = additional_prompt_input if additional_prompt_input else ""

          signal_data = UserDecisionSignal(decision=UserDecision.EDIT, additional_prompt=additional_prompt)
          # TODO Send our Signal to our Workflow Execution we have a handle on
          print("Signal sent to regenerate research")

      else:
          print("Please enter either 'keep', 'edit'")

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "sending_signal_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

## Part 2: Adding Query Support in Temporal

Now let's add **Query** support to our Workflow. Queries allow you to read workflow state without interrupting execution. This can be done during or even after the Workflow Execution. For example, you might want to:

- **Monitor Progress of Long-Running Workflows**: A Client might want to receive updates on the progress, like the percentage of the task completed.
- **Retrieve Results**: Queries can be used to fetch the results of Activities without waiting for the entire Workflow to complete.

In [ ]:
# Stop the current Worker so we can start fresh with Query support
x = worker.cancel()

if x:
  print("Worker killed - ready to start fresh with Query support!")
else:
  print("Worker was not running. Nothing to kill")

## Developing Queries in Temporal

Once a Query is issued, the Client waits for a response from the Workflow. Although Queries are typically used to access the state of an open (running) Workflow Execution, it is also possible to send a Query to a closed Workflow Execution. In either case, there must be at least one running Worker for the Task Queue to which that Workflow belongs.

Your Query should not include any logic that generates commands (such as executing Activities). Remember, Queries are intended to be **read-only operations** that do not alter the Workflow's state.

## Handling a Query

Let's create a Query which will allow external clients to read the current research content from a running Workflow without interrupting its execution.

Similar to Signals, in the Python SDK, you can handle Queries by annotating a function within your Workflow with `@workflow.query`:

```
@workflow.query
def get_research_result(self) -> str | None:
    """Query to get the current research result"""
    return self._research_result
```

In [ ]:
# Run this to kill the current Worker
x = worker.cancel()

if x:
  print("Worker killed")
else:
  print("Worker was not running. Nothing to kill")

### Start a New Workflow Execution

Now that the Worker is running with Query support, start a **new** workflow execution to test Queries.

**Run the cells from Part 1 again (cells 44, 50) but with the new Query-enabled workflow, then run cells 60-61 to send Queries.**

In [ ]:
# Adding a Query into our Workflow code
# Step 1: In your Query handler, returns the `_research_result` field
# Step 2: After the LLM call, set `_research_result` field to be the contents of the research (research_facts["choices"][0]["message"]["content"])
# Step 3: Run this code block to load it into the program
import asyncio
from datetime import timedelta
import logging

from temporalio import workflow

@workflow.defn(sandboxed=False)
class GenerateReportWorkflow:
    def __init__(self) -> None:
        self._current_prompt: str = ""
        # Instance variable to store the Signal in
        self._user_decision: UserDecisionSignal = UserDecisionSignal(decision=UserDecision.WAIT)
        self._research_result: str = ""

    # Method to handle the Signal
    @workflow.signal
    async def user_decision_signal(self, decision_data: UserDecisionSignal) -> None:
        # Update the instance variable with the received Signal data
        self._user_decision = decision_data

    @workflow.query # Query to get the current research result
    def get_research_result(self) -> str | None:
        # TODO: Return the `_research_result` field

    @workflow.run
    async def run(self, input: GenerateReportInput) -> GenerateReportOutput:
        self._current_prompt = input.prompt

        llm_call_input = LLMCallInput(
            prompt=self._current_prompt,
            llm_api_key=input.llm_api_key,
            llm_model=input.llm_research_model,
        )

        continue_agent_loop = True

        while continue_agent_loop:
            research_facts = await workflow.execute_activity(
                llm_call,
                llm_call_input,
                start_to_close_timeout=timedelta(seconds=30),
            )

            # TODO: Set `_research_result` field to be the contents of the research (research_facts["choices"][0]["message"]["content"])

            # Waiting for Signal with user decision
            await workflow.wait_condition(lambda: self._user_decision.decision != UserDecision.WAIT)

            if self._user_decision.decision == UserDecision.KEEP:
                workflow.logger.info("User approved the research. Creating PDF...")
                continue_agent_loop = False
            elif self._user_decision.decision == UserDecision.EDIT:
                workflow.logger.info("User requested research modification.")
                if self._user_decision.additional_prompt != "":
                    self._current_prompt = (
                        f"{self._current_prompt}\n\nAdditional instructions: {self._user_decision.additional_prompt}"
                    )
                    workflow.logger.info(f"Regenerating research with updated prompt: {self._current_prompt}")
                else:
                    workflow.logger.info("No additional instructions provided. Regenerating with original prompt.")
                llm_call_input.prompt = self._current_prompt

                # Set the decision back to WAIT for the next loop
                self._user_decision = UserDecisionSignal(decision=UserDecision.WAIT)

        pdf_generation_input = PDFGenerationInput(content=research_facts["choices"][0]["message"]["content"])

        pdf_filename: str = await workflow.execute_activity(
            create_pdf,
            pdf_generation_input,
            start_to_close_timeout=timedelta(seconds=20),
        )

        return GenerateReportOutput(result=f"Successfully created research report PDF: {pdf_filename}")

In [ ]:
# Run this cell to load and display the solution
from pathlib import Path
from IPython.display import display, Markdown
import os

notebook_dir = Path(os.getcwd())
solution_file = notebook_dir / "Solutions_03_Human_in_the_Loop" / "handling_query_solution.py"

code = solution_file.read_text()

print("Solution loaded:")
display(Markdown(f"```python\n{code}\n```"))

In [ ]:
# Starting the Worker again
worker = asyncio.create_task(run_worker())

 # Check if the task is in the set of all tasks
if worker in asyncio.all_tasks():
    # The sleep is necessary because of the async task scheduling in Jupyter
    print("Task is currently active.") # The Worker now registers the updated Workflow changes
else:
    print("Task is not found in active tasks (might have finished or not yet scheduled).")

## Sending a Query

After defining and setting a handler for the Queries in your Workflow, the next step is to send a Query, which is sent from a Temporal Client. To do this, use the query method. To do this, we will again:

1. Get a handle of the Workflow Execution we will query
2. Send a query with the `query` method.

```python
research_result = await handle.query(GenerateReportWorkflow.get_research_result)
```


In [ ]:
# Let's send our Query from the Client code

async def query_research_result(client: Client, workflow_id: str) -> None:
    handle = client.get_workflow_handle(workflow_id)

    try:
        research_result = await handle.query(GenerateReportWorkflow.get_research_result)
        if research_result:
            print(f"Research Result: {research_result}")
        else:
            print("Research Result: Not yet available")

    except Exception as e:
        print(f"Query failed: {e}")

In [ ]:
# Run this to send your Query
send_query = asyncio.run(query_research_result(client, handle.id))

## Putting it Together

You can now make a research call, check your results by querying your Workflow, then choose to edit or keep the research!

---
# Exercise 3 - Human in the Loop

* In these exercises you will:
  * Review a modified version of the previous exercise and investigate the results in the Web UI
  * Add a Signal to the exercise to provide the filename you wish to save the research report as
  * Add a Query to the exercise to extract the character length of the research request
* Go to the **Exercise** Directory in the Google Drive and open the **Practice** Directory
* Open _01-An-AI-Agent-Practice.ipynb_ and follow the instructions
* If you get stuck, raise your hand and someone will come by and help. You can also check the `Solution` directory for the answers
* **You have 5 mins**